In [1]:
import os
os.chdir("/home/alsun/get/1_methyl_preprocess/")
print(os.getcwd())

/home/alsun/get/1_methyl_preprocess


In [2]:
import os
from pathlib import Path

from gcell._settings import get_setting
from preprocess_utils import (
    add_atpm,
    add_exp,
    create_peak_motif,
    download_motif,
    get_motif,
    query_motif,
)

annotation_dir = Path(get_setting('annotation_dir'))
print("gcell currently using annotation directory:", annotation_dir)

gcell currently using annotation directory: /home/alsun/.gcell_data/annotations


In [3]:
motif_bed_url = "https://resources.altius.org/~jvierstra/projects/motif-clustering/releases/v1.0/hg38.archetype_motifs.v1.0.bed.gz"
motif_bed_index_url = "https://resources.altius.org/~jvierstra/projects/motif-clustering/releases/v1.0/hg38.archetype_motifs.v1.0.bed.gz.tbi"


if (
    motif_bed_url
    and motif_bed_index_url
    and not (
        (annotation_dir / "hg38.archetype_motifs.v1.0.bed.gz").exists()
        or (annotation_dir / "hg38.archetype_motifs.v1.0.bed.gz.tbi").exists()
    )
):
    download_motif(motif_bed_url, motif_bed_index_url, motif_dir=annotation_dir)
    motif_bed = str(annotation_dir / "hg38.archetype_motifs.v1.0.bed.gz")
else:
    motif_bed = str(annotation_dir / "hg38.archetype_motifs.v1.0.bed.gz")

In [8]:
peak_bed = "SAA-125-2M_L2-3.CGN-both.frac.atac.canon.filter.bed" # since all cell types share the same peak set, when querying motifs, we can just use one cell type to query motifs.
peaks_motif = query_motif(peak_bed, motif_bed)
get_motif_output = get_motif(peak_bed, peaks_motif)

Peak motif extraction completed. Results saved in get_motif.bed


In [5]:
get_motif_output

'get_motif.bed'

In [9]:
import zarr
import pandas as pd
import numpy as np
def create_peak_motif(peak_motif_bed, output_zarr, peak_bed):
    """
    Create a peak motif zarr file from a peak motif bed file.

    This function reads a peak motif bed file, pivots the data, and saves it to a zarr file.
    The zarr file contains three datasets: 'data', 'peak_names', 'motif_names', and 'accessibility'.
    The 'data' dataset is a sparse matrix containing the peak motif data.
    The 'peak_names' dataset contains the peak names.
    The 'motif_names' dataset contains the motif names.

    Args:
        peak_motif_bed (str): Path to the peak motif bed file.
        output_zarr (str): Path to the output zarr file.
    """
    import pandas as pd
    motif_annotations = pd.read_excel('https://resources.altius.org/~jvierstra/projects/motif-clustering/releases/v1.0/motif_annotations.xlsx')
    motif_cluster_ids = motif_annotations.Name.unique()
    # Read the peak motif bed file
    peak_motif = pd.read_csv(
        peak_motif_bed,
        sep="\t",
        header=None,
        names=["Chromosome", "Start", "End", "Motif_cluster", "Score"],
    )

    # Pivot the data
    peak_motif_pivoted = peak_motif.pivot_table(
        index=["Chromosome", "Start", "End"],
        columns="Motif_cluster",
        values="Score",
        fill_value=0,
    )

    peak_motif_pivoted.reset_index(inplace=True)
    # add missing motif columns
    for motif_cluster_id in motif_cluster_ids:
        if motif_cluster_id not in peak_motif_pivoted.columns:
            peak_motif_pivoted[motif_cluster_id] = 1
    # Create the 'Name' column
    peak_motif_pivoted["Name"] = peak_motif_pivoted.apply(
        lambda x: f'{x["Chromosome"]}:{x["Start"]}-{x["End"]}', axis=1
    )
    peak_motif_pivoted = peak_motif_pivoted.drop(columns=["Chromosome", "Start", "End"])

    # Read the original peak bed file
    original_peaks = pd.read_csv(
        peak_bed, sep="\t", header=None, names=["Chromosome", "Start", "End", "Score"]
    )

    # exclude chrM and chrY
    original_peaks = original_peaks[~original_peaks.Chromosome.isin(["chrM", "chrY"])]
    original_peaks["Name"] = original_peaks.apply(
        lambda x: f'{x["Chromosome"]}:{x["Start"]}-{x["End"]}', axis=1
    )
    
    new_columns = list(motif_cluster_ids) + ["Name"]

    # sort motif columns
    peak_motif_pivoted = peak_motif_pivoted[new_columns]

    # Merge the pivoted data with the original peaks
    merged_data = pd.merge(original_peaks, peak_motif_pivoted, on="Name", how="left")

    # Fill NaN values with 0 for motif columns
    motif_columns = [
        col
        for col in merged_data.columns
        if col not in ["Chromosome", "Start", "End", "Score", "Name"]
    ]
    
    merged_data[motif_columns] = merged_data[motif_columns].fillna(0)
    peak_length = (merged_data.End - merged_data.Start).values / 400 # convert to kb
    merged_data[motif_columns] = merged_data[motif_columns].div(peak_length, axis=0)
    # Prepare data for zarr storage
    name_values = list(merged_data["Name"].values)
    motif_values = motif_columns

    # Create sparse matrix
    motif_data_matrix = merged_data[motif_columns].values
    # Open zarr store and save data
    from numcodecs import Blosc

    z = zarr.open(output_zarr, mode="w")
    z.create_dataset(
        "data",
        data=motif_data_matrix.data,
        chunks=(1000, motif_data_matrix.shape[1]),
        dtype=np.float32,
        compressor=Blosc(cname="zstd", clevel=3, shuffle=Blosc.BITSHUFFLE),
        shape=motif_data_matrix.shape,
    )
    z.create_dataset("peak_names", data=name_values)
    z.create_dataset("motif_names", data=motif_values)

    print(f"Peak motif data saved to {output_zarr}")

create_peak_motif(get_motif_output, "methyl_atac_peak.zarr", peak_bed) # all cell types will later be added to the same zarr file as we use the same peak set.

Peak motif data saved to methyl_atac_peak.zarr


In [10]:
celltype_for_modeling = ["L2-3"]
for cell_type in celltype_for_modeling:
    add_atpm(
        "methyl_atac_peak.zarr",
        "SAA-125-2M_L2-3.CGN-both.frac.atac.canon.filter.bed",
        cell_type,
    )

       Chromosome      Start        End      aTPM                      Name
0            chr1    3094753    3095675  0.888889      chr1:3094753-3095675
1            chr1    3120133    3120689  1.000000      chr1:3120133-3120689
2            chr1    3121252    3122011  1.000000      chr1:3121252-3122011
3            chr1    3292425    3293285  1.000000      chr1:3292425-3293285
4            chr1    3299335    3300188  0.547619      chr1:3299335-3300188
...           ...        ...        ...       ...                       ...
124973       chrX  168663817  168664678  0.398809  chrX:168663817-168664678
124974       chrX  168672167  168672855  1.000000  chrX:168672167-168672855
124975       chrX  168673441  168674364  0.875000  chrX:168673441-168674364
124976       chrX  168795795  168796578  0.828125  chrX:168795795-168796578
124977       chrX  169299327  169300224  0.000000  chrX:169299327-169300224

[124978 rows x 5 columns]
